In [23]:


import os
import sys
from git import Repo, InvalidGitRepositoryError
from pathlib import Path
import time

In [24]:
def get_repo_url():
    """Get repository URL from user input"""
    while True:
        repo_url = input("Enter the Git repository URL (SSH or HTTPS): ").strip()
        if repo_url:
            return repo_url
        print("Please enter a valid repository URL.")

In [25]:
def initialize_or_clone_repo(repo_url, local_path="."):
    """Initialize or clone the repository"""
    try:
        # Try to open existing repo
        repo = Repo(local_path)
        print(f"Found existing Git repository at {local_path}")
        
        # Add remote if it doesn't exist
        try:
            origin = repo.remote('origin')
            if origin.url != repo_url:
                print(f"Warning: Remote origin URL differs from provided URL")
                print(f"Existing: {origin.url}")
                print(f"Provided: {repo_url}")
        except:
            repo.create_remote('origin', repo_url)
            print(f"Added remote origin: {repo_url}")
            
        return repo
        
    except InvalidGitRepositoryError:
        # Initialize new repo
        print(f"Initializing new Git repository at {local_path}")
        repo = Repo.init(local_path)
        repo.create_remote('origin', repo_url)
        print(f"Created new repository with remote: {repo_url}")
        return repo


In [26]:
def get_all_files_and_folders(path="."):
    """Get all files and folders in the current directory"""
    items = []
    
    # Walk through directory tree
    for root, dirs, files in os.walk(path):
        # Skip .git directory
        if '.git' in root:
            continue
            
        # Add files
        for file in files:
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(file_path, path)
            # Skip hidden files and common unwanted files
            if not relative_path.startswith('.') or relative_path == '.gitignore':
                items.append(('file', relative_path))
    
    return items

In [27]:
def commit_and_push_item(repo, item_type, item_path, branch='main'):
    """Commit and push a single item"""
    try:
        # Check if item still exists
        full_path = Path(repo.working_dir) / item_path
        if not full_path.exists():
            print(f"Skipping {item_path} - file/folder no longer exists")
            return False
        
        # Add the item to staging
        repo.index.add([item_path])
        
        # Check if there are any staged changes
        staged_files = repo.index.diff("HEAD") if repo.head.is_valid() else repo.index.diff(None)
        if not staged_files:
            print(f"No changes to commit for {item_path}")
            return False
        
        # Create commit message
        commit_msg = f"Add {item_type}: {item_path}"
        
        # Commit
        commit = repo.index.commit(commit_msg)
        print(f"Committed {item_type}: {item_path} (commit: {commit.hexsha[:8]})")
        
        # Push to remote
        origin = repo.remote('origin')
        
        # Ensure we're on the correct branch
        try:
            current_branch = repo.active_branch.name
            if current_branch != branch:
                try:
                    repo.git.checkout(branch)
                except:
                    # Create branch if it doesn't exist
                    repo.git.checkout('-b', branch)
        except:
            # For new repos, create the main branch
            repo.git.checkout('-b', branch)
        
        # Push to remote
        origin.push(refspec=f'{branch}:{branch}')
        print(f"Pushed {item_type}: {item_path} to remote/{branch}")
        
        return True
        
    except Exception as e:
        print(f"Error processing {item_path}: {str(e)}")
        return False

In [28]:
def main():
    """Main function"""
    print("Auto Git Commit and Push Script")
    print("=" * 50)
    
    # Get repository URL
    repo_url = get_repo_url()
    
    # Initialize or clone repository
    try:
        repo = initialize_or_clone_repo(repo_url)
    except Exception as e:
        print(f"Error initializing repository: {str(e)}")
        sys.exit(1)
    
    # Get all files and folders
    print("\nDiscovering files and folders...")
    items = get_all_files_and_folders()
    
    if not items:
        print("No files or folders found to commit.")
        return
    
    print(f"Found {len(items)} items to process:")
    for item_type, item_path in items:
        print(f"  - {item_type}: {item_path}")
    
    # Confirm before proceeding
    print(f"\nThis will create {len(items)} separate commits and push them one by one.")
    confirm = input("Do you want to proceed? (y/N): ").strip().lower()
    
    if confirm != 'y':
        print("Operation cancelled.")
        return
    
    # Process each item
    print(f"\nProcessing {len(items)} items...")
    success_count = 0
    
    for i, (item_type, item_path) in enumerate(items, 1):
        print(f"\n[{i}/{len(items)}] Processing {item_type}: {item_path}")
        
        if commit_and_push_item(repo, item_type, item_path):
            success_count += 1
        
        # Small delay between operations to avoid overwhelming the remote
        if i < len(items):
            time.sleep(0.5)
    
    # Summary
    print(f"\nSummary:")
    print(f"  - Successfully processed: {success_count}/{len(items)} items")
    print(f"  - Failed: {len(items) - success_count}/{len(items)} items")
    
    if success_count == len(items):
        print("All files and folders have been successfully committed and pushed!")
    else:
        print("Some items could not be processed. Check the error messages above.")



In [29]:
if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\nOperation cancelled by user.")
        sys.exit(1)
    except Exception as e:
        print(f"\nUnexpected error: {str(e)}")
        sys.exit(1)

Auto Git Commit and Push Script
Found existing Git repository at .

Discovering files and folders...
Found 3 items to process:
  - file: .gitignore
  - file: git_operations.log
  - file: git_push.ipynb

This will create 3 separate commits and push them one by one.
Found existing Git repository at .

Discovering files and folders...
Found 3 items to process:
  - file: .gitignore
  - file: git_operations.log
  - file: git_push.ipynb

This will create 3 separate commits and push them one by one.

Processing 3 items...

[1/3] Processing file: .gitignore
No changes to commit for .gitignore

Processing 3 items...

[1/3] Processing file: .gitignore
No changes to commit for .gitignore

[2/3] Processing file: git_operations.log
No changes to commit for git_operations.log

[2/3] Processing file: git_operations.log
No changes to commit for git_operations.log

[3/3] Processing file: git_push.ipynb
Committed file: git_push.ipynb (commit: 0b97c3dc)

[3/3] Processing file: git_push.ipynb
Committed fi